<a href="https://colab.research.google.com/github/moonlightbotanist/ColoradoView_LST/blob/master/Biophysical_Predictors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Script for Calculating Biophysical Predictors and Extracting Zonal Statistics**

### **Setup Script**

**Import common python libs**

In [0]:
import os
import sys
import datetime
from datetime import date
from datetime import datetime
from datetime import datetime, timedelta
import math
import csv
import numpy as np                  # to create a sequence for plotting
from scipy.spatial import distance  # for Jensen-Shannon
import matplotlib.pyplot as plt     # for plotting histograms
import pandas as pd                 # for creating histogram dataframe to export to GDrive
from google.colab import drive      # for exporting from distributed machine to GDrive
from google.colab import files

**Install, import, & authenticate earthengine python API**

In [0]:
##reference: https:#developers.google.com/earth-engine/python_install_manual

!pip install 'pyOpenSSL>=0.11'
!pip install earthengine-api

     |████████████████████████████████| 61kB 4.1MB/s 
     |████████████████████████████████| 2.3MB 13.5MB/s 


In [0]:
##@title set up authentication credentials (earthengine)
!earthengine authenticate

# test 1: should not show any error message with the following command
#!python -c "import ee; ee.Initialize()"
# Import the Earth Engine Python Package
import ee
# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

# test 2: should print the metadata of the test image
# Print the information for an image asset.
image = ee.Image('srtm90_v4')
print(image.getInfo())


Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/wAEytPXwlayzjoo92L0rmDG9yINZ2xgYgBPIGlibC2I

### **Begin calculating Biophysical Predictors**

**Import Geometries for Great Basin and AIM plots (with buffers)**

In [0]:
GBbounds = ee.FeatureCollection('users/ericjensen41_default/Thesis/Project_Boundaries/LIIIGBBoundary')
AIMplots = ee.FeatureCollection('users/ericjensen41_default/Thesis/Plots/AIM_modelbuff_100m')
print(AIMplots.first().getInfo())
print(GBbounds.getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-117.77100344785721, 42.33769301660359], [-117.77099449914212, 42.337648390820725], [-117.77098561019372, 42.33760382157669], [-117.77097676035179, 42.33755476864474], [-117.7709588689012, 42.33751017856241], [-117.77094106879544, 42.3374655645895], [-117.77092317741756, 42.337420974600846], [-117.77089643463403, 42.33737639625998], [-117.77086974411652, 42.33733627773599], [-117.77083847688296, 42.33729613249026], [-117.77080278400787, 42.33725599306898], [-117.77076709120706, 42.33721585367323], [-117.77072702498008, 42.337180179851806], [-117.77068686752261, 42.3371445299803], [-117.77064227689331, 42.337113289170226], [-117.77059769391177, 42.33707764510292], [-117.77054863863765, 42.33705089376011], [-117.77049510560065, 42.33701968844591], [-117.77044610265801, 42.336997396819925], [-117.7703881365501, 42.33697060050257], [-117.77033461679184, 42.33694833854968], [-117.77027666385476, 42.33693048558522], [-117.

**Import input datasets from Conservation Science partners for calculating values**

In [0]:
# US NED CHILI (Continuous Heat-Insolation Load Index)
csp_chili = ee.Image('CSP/ERGo/1_0/US/CHILI').rename('csp_chili').resample('bicubic')
# US NED mTPI (Multi-Scale Topographic Position Index)
csp_tpi = ee.Image('CSP/ERGo/1_0/US/mTPI').rename('csp_tpi').resample('bicubic')
# US NED Topographic Diversity
csp_topodiv = ee.Image('CSP/ERGo/1_0/US/topoDiversity').rename('csp_topodiv').resample('bicubic')
# US NED Landforms
csp_landforms = ee.Image('CSP/ERGo/1_0/US/landforms').rename('csp_landforms').resample('bicubic')
# US NED Physiographic Diversity
csp_physd = ee.Image('CSP/ERGo/1_0/US/physioDiversity').rename('csp_physd').resample('bicubic')

**Calculate additional biophysical predictors**

In [0]:
# Latitude and longitude
latlong = ee.Image.pixelLonLat(); 

# Topography
elevation = ee.Image('USGS/NED').rename('elev').resample('bicubic')
aspect = ee.Terrain.aspect(elevation).multiply(ee.Number(math.pi).divide(ee.Number(180))).rename('aspect') # linear aspect
northness = aspect.cos().rename('northness')
eastness = aspect.sin().rename('eastness')
slope = ee.Terrain.slope(elevation)

# Stage 1975: slope% * cos or sin of aspect
slope_pct = slope.expression("tan(b(0) * pi/180)", {"pi": ee.Number(math.pi)}).rename("slope_pct")
slope_pct = slope_pct.where(slope_pct.gt(1), 1.01)

slope_east = slope_pct.multiply(eastness).rename('slope_east')
slope_north = slope_pct.multiply(northness).rename('slope_north')

# TRASP - Roberts and Cooper 1989
trasp = aspect.expression("(1-(cos(b(0)-d))) / 2", {"d":(ee.Number(30).multiply(ee.Number(math.pi).divide(ee.Number(180))))}).rename("trasp")

# Topographic position index
tpi90 = elevation.subtract(elevation.focal_mean(radius=90, units='meters')).rename('tpi90')
tpi990 = elevation.subtract(elevation.focal_mean(radius=990, units='meters')).rename('tpi990')

**Create single multi-band image with all of the topographic bands of interest**

In [0]:
# add topographic bands to the image for calculating means
topo_mean = ee.Image.cat(latlong, elevation, aspect, northness, eastness, slope_pct, slope_east, slope_north, trasp, tpi90, tpi990, csp_chili, csp_tpi, csp_topodiv, csp_physd)
# add topographic bands to the image for calculating means
topo_mode = ee.Image.cat(csp_landforms)

# Reference table to landforms dataset
# 11	141414	Peak/ridge (warm)
# 12	383838	Peak/ridge
# 13	808080	Peak/ridge (cool)
# 14	EBEB8F	Mountain/divide
# 15	F7D311	Cliff
# 21	AA0000	Upper slope (warm)
# 22	D89382	Upper slope
# 23	DDC9C9	Upper slope (cool)
# 24	DCCDCE	Upper slope (flat)
# 31	1C6330	Lower slope (warm)
# 32	68AA63	Lower slope
# 33	B5C98E	Lower slope (cool)
# 34	E1F0E5	Lower slope (flat)
# 41	a975ba	Valley
# 42	6f198c	Valley (narrow)

topo_mean_bands = topo_mean.bandNames().getInfo()
print(topo_mean_bands)

topo_mode_bands = topo_mode.bandNames().getInfo()
print(topo_mode_bands)

['longitude', 'latitude', 'elev', 'aspect', 'northness', 'eastness', 'slope_pct', 'slope_east', 'slope_north', 'trasp', 'tpi90', 'tpi990', 'csp_chili', 'csp_tpi', 'csp_topodiv', 'csp_physd', 'csp_mtpi']
['csp_landforms']


### **Calculate zonal statistics and export**

In [0]:
def mean_export(image):
  mean_FC = image.reduceRegions(reducer = ee.Reducer.mean(), collection = AIMplots, scale = 30)
  mean_exp_task = ee.batch.Export.table.toDrive(collection = mean_FC,
                                            description = 'Biophysical_Predictors_Mean', 
                                            fileNamePrefix = 'Biophysical_Predictors_Mean',
                                            fileFormat = 'CSV')
  mean_exp_task.start()

In [0]:
def mode_export(image):
  mode_FC = image.reduceRegions(reducer = ee.Reducer.mode(), collection = AIMplots, scale = 30)
  mode_exp_task = ee.batch.Export.table.toDrive(collection = mode_FC,
                                            description = 'Biophysical_Predictors_Mode', 
                                            fileNamePrefix = 'Biophysical_Predictors_Mode',
                                            fileFormat = 'CSV')
  mode_exp_task.start()

In [0]:
mean_export(topo_mean)

In [0]:
mode_export(topo_mode)